# Install all the necessary libraries for running the scripts

In [1]:
!pip install pandas
!pip install numpy
!pip install pandas geopy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Make sure the data is in the right format for geocoding

In [2]:
import pandas as pd
import re

## First create a new full address column

In [8]:
df = pd.read_csv('D:/GitHub/CDC-ATSDR-SVI-Match/data/sample_address_data.csv')

df['FULL_ADDRESS'] = df['ADDRESS'] + ", " + df['CITY'] + ", " + df['STATE'] + " " + df['ZIP CODE'].astype(str)
print(df['FULL_ADDRESS'].head())

NameError: name 'df' is not defined

In [3]:
def clean_address(address):
    """Clean and standardize the address."""

    # Convert to title case
    address = address.title()

    # Remove any leading or trailing white spaces
    address = address.strip()

    # Replace multiple spaces with a single space
    address = re.sub(r'\s+', ' ', address)

    # Standardize common abbreviations
    abbreviations = {
        ' Street': ' St',
        ' Avenue': ' Ave',
        ' Boulevard': ' Blvd',
        ' Drive': ' Dr',
        ' Road': ' Rd',
        ' Lane': ' Ln',
        ' Court': ' Ct',
        ' Place': ' Pl',
        ' Square': ' Sq',
        ' Loop': ' Lp',
        ' Highway': ' Hwy',
        ' Terrace': ' Ter',
        ' Parkway': ' Pkwy',
        ' Suite': ' Ste',
        ' Apartment': ' Apt'
    }
    
    for long, short in abbreviations.items():
        address = address.replace(long, short)

    # Remove special characters except for # which can denote apt/suite numbers
    address = re.sub(r'[!@&$%^*()_+={}\[\]:;<>,.?~\\]', '', address)
    
    return address

In [6]:
def main():
    # Load the address dataset
    df = pd.read_csv('D:/GitHub/CDC-ATSDR-SVI-Match/data/sample_address_data.csv')

    # Clean the address column
    df['ADDRESS'] = df['ADDRESS'].apply(clean_address)

    # Save the cleaned dataframe to a new CSV file
    df.to_csv('D:/GitHub/CDC-ATSDR-SVI-Match/data/cleaned_sample_address_data.csv', index=False)
    print("Address cleaning completed. Data saved to 'D:/GitHub/CDC-ATSDR-SVI-Match/data/cleaned_sample_address_data.csv'.")


In [7]:
if __name__ == "__main__":
    main()

Address cleaning completed. Data saved to 'D:/GitHub/CDC-ATSDR-SVI-Match/data/cleaned_sample_address_data.csv'.


# Geocode the addresses to find the county

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

In [3]:
def get_county_from_address(address):
    """Fetch county from address using Nominatim API."""
    try:
        geolocator = Nominatim(user_agent="geoapiExercises")
        location = geolocator.geocode(address, timeout=10)
        
        if location:
            # Split the address components
            address_components = location.address.split(", ")
            for component in address_components:
                if "County" in component:
                    return component
        return None
    except:
        print(f"Error geocoding address: {address}")
        return None

In [6]:
def main():
    # Load the address dataset
    df = pd.read_csv('D:/GitHub/CDC-ATSDR-SVI-Match/data/sample_address_data.csv')

    # Add a new column for county
    df['COUNTY'] = df['ADDRESS'].apply(get_county_from_address)

    # Save the updated dataframe to a new CSV file
    df.to_csv('sample_address_data_with_county.csv', index=False)
    print("Geocoding completed. Data saved to 'sample_address_data_with_county.csv'.")

In [ ]:
if __name__ == "__main__":
    main()

# Match the county and zip to the fips code

# Finally merge the two dataframes